In [1]:
import numpy as np
import matplotlib.pyplot as plt

import cv2
import os

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

images = load_images_from_folder('dataset')

In [3]:
K = np.loadtxt('intrinsic_matrix.txt', dtype=float)

In [4]:
# def load_data(path='cameras'):

#     C1 = np.load(f'{path}/cam1.npz')
#     C2 = np.load(f'{path}/cam2.npz')
#     K1 = C1['K']
#     K2 = C2['K']

#     return K1

# K = load_data()

In [5]:
# Feature extraction

orb = cv2.ORB_create()

kp, des = [], []
for im in images:
    kp_tmp, des_tmp = orb.detectAndCompute(im, None)
    kp.append(kp_tmp)
    des.append(des_tmp)
kp = np.array(kp) #kp.shape = n_images x n_keypoints
des = np.array(des) #des.shape = n_images x n_keypoints x descriptor_size

In [6]:
# Match features
matches = []

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
for i in range(des.shape[0] - 1):
    matches_tmp = bf.match(des[i], des[i+1])
    matches_tmp = sorted(matches_tmp, key = lambda x:x.distance)
    matches.append(matches_tmp)

# Matches, list of n_images - 1 x num_matching_points 

In [7]:
# Fundamental Matrix

def normalize_points(pts):
    centroid = np.mean(pts, axis=0)
    pts = pts-centroid
    scale=np.sqrt(2)/np.mean(np.sqrt(np.sum(pts**2, axis=1)))
    pts = pts * scale
    T = np.array([[scale, 0, -scale*centroid[0]], [0, scale, -scale*centroid[1]], [0, 0, 1]])
    return pts, T

def eight_point_algorithm(pts1, pts2):
    # Normalize the points
    pts1, T1 = normalize_points(pts1)
    pts2, T2 = normalize_points(pts2)

    # Create constraint matrix A (Af=0), each A element is [(xi, xi),(xi)
    A = np.zeros((len(pts1), 9))
    for i in range(len(pts1)):
        A[i] = [pts1[i,0]*pts2[i,0], pts1[i,1]*pts2[i,0], 1*pts2[i,0],
                pts1[i,0]*pts2[i,1], pts1[i,1]*pts2[i,1], 1*pts2[i,1],
                pts1[i,0]*1,            pts1[i,1]*2,         1*1]
    
    # f is equal to last column of V from svd of A
    _, _, V = np.linalg.svd(A)
    F = np.reshape(V[-1], (3, 3))

    # Enforce rank 2 constraint F=U * diag(S) * V.T
    U, S, V = np.linalg.svd(F)
    S[-1] = 0
    F = U @ np.diag(S) @ V.T

    return F

fundamental_matrices = []
for i in range(kp.shape[0] - 1):
    pts1 = np.array([kp[i][m.queryIdx].pt for m in matches[i]])
    pts2 = np.array([kp[i+1][m.trainIdx].pt for m in matches[i]])
    F = eight_point_algorithm(pts1, pts2)
    fundamental_matrices.append(F)
fundamental_matrices = np.array(fundamental_matrices) # fundamental_matrices.shape = n_images-1[skipped first image] x (3x3 -> F matrix)

In [8]:
# Essential matrix
def essential_from_fundamental(K, F):
    return K.T @ F @ K

essential_matrices = []
for F in fundamental_matrices:
    E = essential_from_fundamental(K, F)
    essential_matrices.append(E)
essential_matrices = np.array(essential_matrices) # essential_matrices.shape = n_images-1[skipping first image] x (3x3 -> E matrix)

In [9]:
# First camera initializes at world center aligned
Ps = np.zeros((len(images), 3, 4))
Ps[0] = K @ np.hstack((np.identity(3), np.zeros(3)[:, np.newaxis]))

In [10]:
# Get Rotation and Translation

def get_rot_trans(E):
    U,S,V = np.linalg.svd(E)
    m = S[:2].mean()
    E = U @ np.array([[m, 0, 0], [0, m, 0], [0, 0, 0]]) @ V
    U,S,V = np.linalg.svd(E)
    W = np.array([[0,-1,0],[1,0,0],[0,0,1]])

    RTs = np.zeros([3, 4, 4])
    RTs[:, :, 0] = np.concatenate([U @ W @ V, U[:, 2].reshape([-1, 1]) / abs(U[:, 2]).max()], axis=1)
    RTs[:, :, 1] = np.concatenate([U @ W @ V, -U[:, 2].reshape([-1, 1]) / abs(U[:, 2]).max()], axis=1)
    RTs[:, :, 2] = np.concatenate([U @ W.T @ V, U[:, 2].reshape([-1, 1]) / abs(U[:, 2]).max()], axis=1)
    RTs[:, :, 3] = np.concatenate([U @ W.T @ V, -U[:, 2].reshape([-1, 1]) / abs(U[:, 2]).max()], axis=1)
    return RTs

rot_trans_list = []
for E in essential_matrices:
    RTs = get_rot_trans(E)
    rot_trans_list.append(RTs)
rot_trans_list = np.array(rot_trans_list) # rot_trans_list.shape = n_images - 1 x (3x4 -> [R|t]) x 4 possibilities

In [11]:
# Choose matrix that has all points in front and minimizes reprojection loss

def triangulate(pts1, pts2, P1, P2):
    # Calculate 3D point
    pts4D = np.zeros((4, pts1.shape[1]))
    pts4D = cv2.triangulatePoints(P1, P2, pts1, pts2, pts4D)
    pts4D /= pts4D[-1, :]

    # Calculate reprojection error
    # First, get reprojections
    pts1_reproj = P1 @ pts4D
    pts2_reproj = P2 @ pts4D

    pts1_reproj /= pts1_reproj[-1, :]
    pts2_reproj /= pts2_reproj[-1, :]

    # Second, homogenize every point to compare with the reprojection
    # pts1 (2xN)
    pts1_homo = np.concatenate((pts1, np.ones((1, pts1.shape[1]))), axis=0)
    pts2_homo = np.concatenate((pts2, np.ones((1, pts2.shape[1]))), axis=0)

    err1 = np.sum(np.square(pts1_reproj - pts1_homo))
    err2 = np.sum(np.square(pts2_reproj - pts2_homo))

    err = err1 + err2
    return pts4D[:3, :], err

for i in range(rot_trans_list.shape[0]):
    RT = rot_trans_list[i]
    best_RT = np.zeros((3, 4))
    best_error = np.finfo('float').max
    for j in range(RTs.shape[2]):
        RT = RTs[:, :, j]
        P2_tmp = K @ RT
        pts1 = np.array([kp[i][m.queryIdx].pt for m in matches[i]]).astype('float').T
        pts2 = np.array([kp[i+1][m.trainIdx].pt for m in matches[i]]).astype('float').T
        X, err = triangulate(pts1, pts2, Ps[i], P2_tmp)
        if err < best_error and np.all(X[-1, :] >= 0):
            best_error = err
            P = X
            Ps[i+1] = P2_tmp
            best_RT = RT

Ps = np.array(Ps)

C:\Users\adria\AppData\Local\Temp\ipykernel_8668\2346666757.py:7: RuntimeWarning: divide by zero encountered in divide
  pts4D /= pts4D[-1, :]
C:\Users\adria\AppData\Local\Temp\ipykernel_8668\2346666757.py:7: RuntimeWarning: invalid value encountered in divide
  pts4D /= pts4D[-1, :]
C:\Users\adria\AppData\Local\Temp\ipykernel_8668\2346666757.py:11: RuntimeWarning: invalid value encountered in matmul
  pts1_reproj = P1 @ pts4D
C:\Users\adria\AppData\Local\Temp\ipykernel_8668\2346666757.py:12: RuntimeWarning: invalid value encountered in matmul
  pts2_reproj = P2 @ pts4D
C:\Users\adria\AppData\Local\Temp\ipykernel_8668\2346666757.py:14: RuntimeWarning: divide by zero encountered in divide
  pts1_reproj /= pts1_reproj[-1, :]
C:\Users\adria\AppData\Local\Temp\ipykernel_8668\2346666757.py:14: RuntimeWarning: invalid value encountered in divide
  pts1_reproj /= pts1_reproj[-1, :]
C:\Users\adria\AppData\Local\Temp\ipykernel_8668\2346666757.py:15: RuntimeWarning: divide by zero encountered in